# 4 Gram Analysis
get every word generated by a method and its frequency. 
Get the 4grams of each of the words and their frequencies. Can then make these plots. 

In [18]:
import pickle
import gzip
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
import pandas as pd    
import torch

n_gram = 4
prompt_length = 100
num_batches = 4

In [19]:
# used to decode:
from decodeLogits import *
vals_dict = {'tfs':[0.25, 0.75, 0.9, 0.95, 0.99], 'flat':[0.01, 0.02, 0.05],
'n': [0.1, 0.25, 0.5, 0.75, 0.9], 'k':[1,10,40,200]  }

In [20]:
for key, params in vals_dict.items():
    print('Key is:', key)
    for par in params:
        if par ==None:
            par = "None"
        print('opening file:', 'gpt-2_output/all_logits_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts.pickle.gz')
        #all_logits = pickle.load( gzip.open('gpt-2_output/all_logits_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts.pickle.gz', 'rb'))
        text = pickle.load( gzip.open('gpt-2_output/all_text_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts.pickle.gz', 'rb'))
        break
    break
        

Key is: tfs
opening file: gpt-2_output/all_logits_tfs-sampling-type_0.25-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts.pickle.gz


In [21]:
text[0].shape

(25, 250)

In [23]:
tot_num = 0
for i in range(num_batches):
    tot_num+= text[i].shape[0]
print('total number of samples:', tot_num)

total number of samples: 100


# works for all but the generated text. 

In [4]:
import matplotlib.pyplot as plt
from scipy.special import softmax
import pandas as pd    
import torch

num_samples = tot_num
num_batches = num_samples//batch_size

word_frequency_for_all = dict()

num_target_too_short = 0

for key, params in vals_dict.items():
    print('Key is:', key)
    for par in params:
        if par ==None:
            par = "None"
        print('opening file:', 'gpt-2_output/all_logits_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts.pickle.gz')
        #all_logits = pickle.load( gzip.open('gpt-2_output/all_logits_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts.pickle.gz', 'rb'))
        text = pickle.load( gzip.open('gpt-2_output/all_text_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts.pickle.gz', 'rb'))
        #rand_selects = pickle.load( gzip.open('gpt-2_output/prompt_rand_selections_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts.pickle.gz', 'rb'))
        #temp_tail_cdfs = np.zeros([num_samples, all_logits[0].shape[2]])
        #temp_tail_ids = np.zeros([num_samples, all_logits[0].shape[2]])
        p_ind = 0
        word_freq_temp = dict()
        for batch in range(num_batches):
            for ind in range(0,text[0].shape[0]):
                
                if p_ind%25 == 0:
                    print('index', str(p_ind))
                    
                # this is the ground truth calculations =====

                decoded_text_generated = decoder_text(text[batch][ind, prompt_length:])
                
                word_set = list(set(decoded_text_generated.split(' '))) # may be some fuction that does this
                for w in word_set:
                    # count the number of occurences.
                    pass
                    #word_freq_temp[]
                
                word_frequency_for_all[key+'-sampling-type_'+str(par)+'prompt_'+str(p_ind)] = ground_token_in_cut_temp
                
                p_ind+=1
                
                break
            break
        break
    break


NameError: name 'tot_num' is not defined

In [ ]:
key+'-sampling-type_'+str(par)+'prompt_'+str(p_ind)]

for key, prompt_dicts in word_frequency_for_all.items():
    